In [ ]:
## 创建模拟数据

将模拟5台设备（DEV-101 至 DEV-105）在30天内、每分钟记录一次的振动数据，包含以下字段：
1. timestamp: 时间戳（主键）
2. device_id: 设备编号
3. vibration_x/y/z: 三轴振动速度（mm/s）——核心分析指标
4. temperature: 设备温度（℃）——相关工况参数
5. load: 设备负载率（%）——相关工况参数

In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

# 1. 基础参数设置
np.random.seed(42)  # 固定随机种子，确保每次生成的数据相同
num_devices = 5
device_ids = [f'DEV-{101 + i}' for i in range(num_devices)]
days = 30
freq = 'T'  # 分钟频率
total_records = num_devices * days * 24 * 60  # 5设备 * 30天 * 24小时 * 60分钟
print(f"将生成总计约 {total_records:,} 条记录...")

# 2. 生成时间序列（最近30天，每分钟一个点）
base_time = datetime.now().replace(microsecond=0) - timedelta(days=days)
timestamps = pd.date_range(start=base_time, periods=days*24*60, freq=freq)

# 3. 定义生成单台设备数据的函数
def generate_device_data(device_id, base_timestamps):
    """为核心设备生成带模式、噪声和异常的数据"""
    num_points = len(base_timestamps)
    
    # 基础趋势：设备间有轻微差异
    if device_id == 'DEV-101':
        trend = np.linspace(0, 0.5, num_points)  # 轻微上升趋势
    elif device_id == 'DEV-104':
        trend = np.linspace(0, 1.0, num_points)  # 明显上升趋势（模拟潜在问题）
    else:
        trend = np.linspace(0, 0.2, num_points)  # 平缓趋势
    
    # 周期性模式：模拟日间/夜间工作模式 (24小时周期)
    hours = base_timestamps.hour.values
    # 白天（8-18点）振动和温度更高
    daily_cycle = np.sin(2 * np.pi * hours / 24) * 2
    
    # 3.1 生成振动数据 (vibration_x, y, z) - 它们是相关的但有差异
    # 基础值 + 趋势 + 日周期 + 随机噪声 + 突发异常
    base_vib_x = 5.0 + trend + daily_cycle * 0.5 + np.random.normal(0, 0.8, num_points)
    base_vib_y = 4.5 + trend * 0.9 + daily_cycle * 0.4 + np.random.normal(0, 0.7, num_points)
    base_vib_z = 6.0 + trend * 1.1 + daily_cycle * 0.6 + np.random.normal(0, 1.0, num_points)
    
    # 3.2 添加偶发的异常尖峰（模拟设备瞬态异常）
    spike_indices = np.random.choice(num_points, size=num_points//200, replace=False)  # 约0.5%的异常点
    spike_magnitude = np.random.uniform(10, 20, len(spike_indices))
    base_vib_x[spike_indices] += spike_magnitude
    base_vib_y[spike_indices] += spike_magnitude * 0.8
    base_vib_z[spike_indices] += spike_magnitude * 1.2
    
    # 3.3 生成相关的工况数据：温度与负载
    # 温度与振动正相关，但有滞后和噪声
    temperature = 65.0 + trend * 3 + daily_cycle * 5 + np.random.normal(0, 2, num_points)
    # 负载率：80%基础，加上日波动和随机变化
    load = 80.0 + daily_cycle * 10 + np.random.normal(0, 5, num_points)
    load = np.clip(load, 40, 100)  # 限制在40%-100%之间
    
    # 3.4 合并数据
    device_data = pd.DataFrame({
        'timestamp': base_timestamps,
        'device_id': device_id,
        'vibration_x': np.round(base_vib_x, 3),
        'vibration_y': np.round(base_vib_y, 3),
        'vibration_z': np.round(base_vib_z, 3),
        'temperature': np.round(temperature, 1),
        'load': np.round(load, 1)
    })
    
    return device_data

# 4. 循环生成所有设备数据并合并
print("正在生成各设备数据...")
all_device_dfs = []
for dev_id in device_ids:
    dev_df = generate_device_data(dev_id, timestamps)
    all_device_dfs.append(dev_df)
    print(f"  已生成 {dev_id} 的数据，形状: {dev_df.shape}")

# 合并所有设备数据
df_combined = pd.concat(all_device_dfs, ignore_index=True)

# 5. 故意插入一些缺失值 (NaN) 以模拟真实数据不完整性
missing_mask = np.random.random(len(df_combined)) < 0.005  # 约0.5%的数据随机缺失
df_combined.loc[missing_mask, 'vibration_z'] = np.nan

# 特定设备在某时间段数据缺失（模拟传感器故障）
dev104_mask = (df_combined['device_id'] == 'DEV-104') & \
              (df_combined['timestamp'] > pd.Timestamp.now() - timedelta(days=5))
df_combined.loc[dev104_mask, 'temperature'] = np.nan

print(f"\n合并后总数据形状: {df_combined.shape}")
print(f"时间范围: {df_combined['timestamp'].min()} 至 {df_combined['timestamp'].max()}")

# 6. 保存数据到项目 raw 文件夹
project_root = os.path.abspath(os.path.join('..'))
raw_data_dir = os.path.join(project_root, 'data', 'raw')
os.makedirs(raw_data_dir, exist_ok=True)  # 确保目录存在

output_path = os.path.join(raw_data_dir, 'vibration_data_2023Q4.csv')
df_combined.to_csv(output_path, index=False)
print(f"\n✅ 模拟数据已成功保存至: {output_path}")
print(f"文件大小: {os.path.getsize(output_path) / (1024*1024):.2f} MB")

# 7. 数据预览
print("\n数据预览（前10行）:")
print(df_combined.head(10))
print("\n各设备数据量统计:")
print(df_combined['device_id'].value_counts().sort_index())
print("\n各列数据类型:")
print(df_combined.dtypes)